In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
import plotly.express as px
from keras.layers.convolutional import MaxPooling1D, Conv1D
from tensorflow.keras.layers import Dense, Dropout, Flatten,LSTM, TimeDistributed, ConvLSTM2D

from keras.regularizers import l1
from keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

2023-03-30 16:04:45.156826: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 16:04:45.201508: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 16:04:45.202534: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 16:04:46.258938: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import glob
all_f= glob.glob('../'+'DataSets/'+'*/'+'*/'+'*_INDEX_*.csv')
all_f

['../DataSets/RAJASTHAN/BHIWADI/H_Bhiw_1_12_17-2_12_22-*01* (copy)_INDEX_Mean.csv',
 '../DataSets/RAJASTHAN/JODHPUR/H_Jodh_1_12_15-2_12_22-*10* (copy)_INDEX_Mean.csv',
 '../DataSets/MADHYA_PRADESH/SINGRAULI/H_Sing_1_12_17-2_12_22-*48* (copy)_INDEX_Mean.csv',
 '../DataSets/GUJARAT/ANKLESHWAR/H_Ankl_1_2_19-3_12_22-*41* (copy)_INDEX_Mean.csv',
 '../DataSets/PUNJAB/LUDHIANA/H_Ludh_1_5_17-_2_12_22-*07* (copy)_INDEX_Mean.csv',
 '../DataSets/WEST_BENGAL/DURGAPUR/H_Durg_1_12_17-2_12_22-*58* (copy)_INDEX_Median.csv',
 '../DataSets/HARYANA/YAMUNA_NAGAR/H_Yamu_1_1_19-2_12_22-*30* (copy)_INDEX_Mean.csv',
 '../DataSets/HARYANA/CHARKHI_DADRI/H_Char_1_3_20-2_12_22-*58* (copy)_INDEX_Mean.csv',
 '../DataSets/HARYANA/JIND/H_Jind_1_1_19-2_12_22-*56* (copy)_INDEX_Median.csv',
 '../DataSets/HARYANA/KURUKSHETRA/H_Kuru_1_1_19-2_12_12-*48* (copy)_INDEX_Mean.csv',
 '../DataSets/HARYANA/SONIPAT/H_Soni_1_1_19-2_12_22-*21* (copy)_INDEX_Mean.csv',
 '../DataSets/HARYANA/DHARUHERA/H_Dhar_1_1_19-2_12_22-*43* (copy)_I

In [3]:
li=[]
for file_name in all_f:
    df=pd.read_csv(file_name)
    li.append(df)


In [4]:
s_n=1
df=li[s_n]
df,all_f[s_n]

(                      DATE  PM2.5
 0      2015-12-01 00:00:00  52.97
 1      2015-12-01 01:00:00  50.10
 2      2015-12-01 02:00:00  53.32
 3      2015-12-01 03:00:00  55.84
 4      2015-12-01 04:00:00  65.57
 ...                    ...    ...
 61404  2022-12-02 12:00:00  75.94
 61405  2022-12-02 13:00:00  58.98
 61406  2022-12-02 14:00:00  48.72
 61407  2022-12-02 15:00:00  36.51
 61408  2022-12-02 16:00:00  18.57
 
 [61409 rows x 2 columns],
 '../DataSets/RAJASTHAN/JODHPUR/H_Jodh_1_12_15-2_12_22-*10* (copy)_INDEX_Mean.csv')

In [5]:
#df = pd.read_csv('../../../../GitHub/Ds/PUNJAB/LUDHIANA/H_Ludh_1_5_17-_2_12_22-*07* (copy)_INDEX_Mean.csv')
#df
v=all_f[s_n].split(r"/")
#u=v[3].split(r' ')
Pre_Ds=v[0]+'/'+'Results/'+v[2]+'/'+v[3]+'/'
Pre_Ds

'../Results/RAJASTHAN/JODHPUR/'

In [6]:
df=df.iloc[16188:,:]
df['DATE']=pd.to_datetime(df['DATE'])
df=df.rename(columns={'PM2.5':'PM'})
df['PM']=df['PM'].astype('float32')
df=df.reset_index()
df=df.drop(['index'], axis = 1) 
#df.drop(['level_0'], axis = 1, inplace = True) 
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45221 entries, 0 to 45220
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    45221 non-null  datetime64[ns]
 1   PM      45221 non-null  float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 530.1 KB


/tmp/ipykernel_6572/17490605.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE']=pd.to_datetime(df['DATE'])


,DATE,PM
0,2017-10-05 12:00:00,56.340000
1,2017-10-05 13:00:00,68.309998
2,2017-10-05 14:00:00,57.869999
3,2017-10-05 15:00:00,57.389999
4,2017-10-05 16:00:00,55.330002
...,...,...
45216,2022-12-02 12:00:00,75.940002
45217,2022-12-02 13:00:00,58.980000
45218,2022-12-02 14:00:00,48.720001
45219,2022-12-02 15:00:00,36.509998


In [7]:
df.set_index('DATE',inplace = True)

In [8]:
df.shape

(45221, 1)

In [9]:
values=df.values
scaler=MinMaxScaler(feature_range=(0,1))
scaled=scaler.fit_transform(values)

In [10]:
train,test=train_test_split(scaled,test_size=0.30,shuffle=False)
train_target=train[:]
test_target=test[:]
train_target.shape,test_target.shape

((31654, 1), (13567, 1))

In [11]:
n_input = 24
n_features =1
b_size=10
train_generator = TimeseriesGenerator(train, train_target, length = n_input, batch_size=b_size)
print(len(train),len(train_generator))

x,y=train_generator[50]

test_genrator = TimeseriesGenerator(test,test_target,length= n_input,batch_size=b_size)
print(len(test),len(test_genrator))
test_genrator[0][0],test_genrator[0][1]

x,y = test_genrator[0]

print(x.shape,y.shape,test.shape)

31654 3163
13567 1355
(10, 24, 1) (10, 1) (13567, 1)


In [12]:
# attention define by the machine learning mastery
import keras.backend as K
from keras.layers import Layer

# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [13]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [14]:
def my_checkpoint(Model):
    mc = ModelCheckpoint('Model_' +'best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    return mc

In [15]:
mc=my_checkpoint(LSTM)

In [16]:
model = tf.keras.models.Sequential()

In [18]:
model.add(Conv1D(filters=256, kernel_size=5, activation='relu', input_shape=(n_input, n_features), bias_regularizer=l1(0.4)))
#model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=1))
#model.add(Flatten())
#model.add(TimeDistributed(Flatten()))
model.add(LSTM(256, activation='relu', return_sequences=False, bias_regularizer=l1(0.4)))
#model.add(Dropout(0.4))
#model.add(LSTM(75, activation='relu', bias_regularizer=l1(0.01)))
#model.add(Dropout(0.3))
#model.add(Dense(1))
model.compile(optimizer='rmsprop', loss='mse')

model.summary()

ValueError: Input 0 of layer "conv1d_1" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 256)